**Importing Pacakges**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Activation

from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint

from keras.optimizers import RMSprop

import random
import sys

from pprint import pprint

In [ ]:
poem = '''
Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you like your blueeyed boy
Mister Death

Had I the heaven’s embroidered cloths,
Enwrought with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and the half-light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spread my dreams under your feet;
Tread softly because you tread on my dreams.

He clasps the crag with crooked hands;
Close to the sun in lonely lands,
Ring’d with the azure world, he stands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbolt he falls.

Some say the world will end in fire,
Some say in ice.
From what I’ve tasted of desire
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To say that for destruction ice
Is also great
And would suffice.

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day!
Yet knowing how way leads on to way,
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I—
I took the one less traveled by,
And that has made all the difference.
'''

In [ ]:
len(poem)

1759

**Poem char and Index Mapping**

In [ ]:
vocab = sorted(list(set(poem)))

char_to_indices  = dict((c,i) for i,c in enumerate(vocab))
indices_to_char = dict((i,c) for i,c in enumerate(vocab))

In [ ]:
pprint(vocab)

['\n',
 ' ',
 '!',
 ',',
 '-',
 '.',
 ':',
 ';',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'H',
 'I',
 'J',
 'M',
 'O',
 'R',
 'S',
 'T',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 '—',
 '’']


In [ ]:
pprint(char_to_indices)
pprint(indices_to_char)

{'\n': 0,
 ' ': 1,
 '!': 2,
 ',': 3,
 '-': 4,
 '.': 5,
 ':': 6,
 ';': 7,
 'A': 8,
 'B': 9,
 'C': 10,
 'D': 11,
 'E': 12,
 'F': 13,
 'H': 14,
 'I': 15,
 'J': 16,
 'M': 17,
 'O': 18,
 'R': 19,
 'S': 20,
 'T': 21,
 'Y': 22,
 'a': 23,
 'b': 24,
 'c': 25,
 'd': 26,
 'e': 27,
 'f': 28,
 'g': 29,
 'h': 30,
 'i': 31,
 'j': 32,
 'k': 33,
 'l': 34,
 'm': 35,
 'n': 36,
 'o': 37,
 'p': 38,
 'q': 39,
 'r': 40,
 's': 41,
 't': 42,
 'u': 43,
 'v': 44,
 'w': 45,
 'y': 46,
 'z': 47,
 '—': 48,
 '’': 49}
{0: '\n',
 1: ' ',
 2: '!',
 3: ',',
 4: '-',
 5: '.',
 6: ':',
 7: ';',
 8: 'A',
 9: 'B',
 10: 'C',
 11: 'D',
 12: 'E',
 13: 'F',
 14: 'H',
 15: 'I',
 16: 'J',
 17: 'M',
 18: 'O',
 19: 'R',
 20: 'S',
 21: 'T',
 22: 'Y',
 23: 'a',
 24: 'b',
 25: 'c',
 26: 'd',
 27: 'e',
 28: 'f',
 29: 'g',
 30: 'h',
 31: 'i',
 32: 'j',
 33: 'k',
 34: 'l',
 35: 'm',
 36: 'n',
 37: 'o',
 38: 'p',
 39: 'q',
 40: 'r',
 41: 's',
 42: 't',
 43: 'u',
 44: 'v',
 45: 'w',
 46: 'y',
 47: 'z',
 48: '—',
 49: '’'}


**Pre-Processing the Data**

In [ ]:
max_length = 100
steps = 5

sentences = []
next_char = []

for i in range(0,len(poem)-max_length-1,steps):
  sentences.append(poem[i:i+max_length])
  next_char.append(poem[i+max_length])
print("Training Data")
for i,j in zip(sentences,next_char):
  pprint(f"Sentences: {i} Next Character:  {j}")

Training Data
('Sentences: \n'
 'Buffalo Bill’s\n'
 'defunct\n'
 'who used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four  Next Character:  f')
('Sentences: alo Bill’s\n'
 'defunct\n'
 'who used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four five  Next Character:  p')
('Sentences: ill’s\n'
 'defunct\n'
 'who used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four five pigeo Next Character:  n')
('Sentences: \n'
 'defunct\n'
 'who used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four five pigeons ju Next Character:  s')
('Sentences: nct\n'
 'who used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four five pigeons just li Next Character:  k')
('Sentences: ho used to\n'
 'ride a watersmooth-silver\n'
 'stallion\n'
 'and break one two three four five pigeons just like th Next Character:  a')
('Sentences: ed to\n'
 'ride a water

In [ ]:
X = np.zeros((len(sentences),max_length,len(vocab)),dtype=np.bool)
y = np.zeros((len(sentences),len(vocab)),dtype=np.bool)

<ipython-input-30-b4735cae025a>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences),max_length,len(vocab)),dtype=np.bool)
<ipython-input-30-b4735cae025a>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences),len(vocab)),dtype=np.bool)


In [ ]:
for i,sentence in enumerate(sentences):
  for t,char in enumerate(sentence):
    X[i,t,char_to_indices[char]] = 1
  y[i,char_to_indices[next_char[i]]] = 1

**Building Model**

In [ ]:
model = Sequential()

model.add(GRU(128,input_shape=(max_length,len(vocab))))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))

optimizers = RMSprop(learning_rate=0.01)

model.compile(loss = 'categorical_crossentropy' , optimizer=optimizers)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 128)               69120     
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
Total params: 75570 (295.20 KB)
Trainable params: 75570 (295.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Next Sample Index**

In [ ]:
def sample_index(preds,temparture=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temparture
  exp_pred = np.exp(preds)
  preds = exp_pred / np.sum(exp_pred)

  probs = np.random.multinomial(1,preds,1)
  return np.argmax(probs)

**On Each Epoch End**

In [ ]:
def on_epoch_end(epoch,logs):
  print()
  print(f"Running at Epoch: {epoch}")

  start_index = random.randint(0,len(poem)-max_length-1)

  for diversity in [0.2,0.5,1.0,1.2]:

    print(f"Diversity: {diversity}")

    generated = ""
    sentence = poem[start_index: start_index+max_length]
    generated += sentence

    sys.stdout.write(generated)

    for i in range(400):
      x_pred = np.zeros((1,max_length,len(vocab)))

      for t,char in enumerate(sentence):
        x_pred[0,t,char_to_indices[char]] = 1

      preds = model.predict(x_pred,verbose=0)[0]
      next_index = sample_index(preds,diversity)
      next_char = indices_to_char[next_index]

      generated += next_char
      sentence = sentence[1:]+next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()

    print()

print_callback = LambdaCallback(on_epoch_end = on_epoch_end)

**Keeping Model Check Points**

In [ ]:
?ModelCheckpoint

In [ ]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath = filepath, save_weights_only=True,
                             monitor='loss',save_best_only=True,mode='min')

**Defining Callbacks Together**

In [ ]:
callbacks= [print_callback,checkpoint]

**Fiting the Model**

In [ ]:
model.fit(X,y,epochs=15,batch_size=128,callbacks=callbacks)

Epoch 1/15
3/3 [==============================] - ETA: 0s - loss: 3.9644
Running at Epoch: 0
Diversity: 0.2
e
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To rarsysoeettJIcejn.er’linlpwsu’myokn!ealMaMrshrent taeedneezfuehtasnTedsBMsmraDphrahmmmnArefdnsaMuBlelOnheqJoo-sotderaafpsO-e
szaenCyftAngca deeealsdp,emd,
aandsncMeassfBrv!YunluzoaiheRes:—ceFTykhakeTnhaiM;ibmnniadahemyeaatAFMT.msznaun;daHfaAHoaejaranM
dTdD
ec,:Coaatevhaifn;mnkcmdMse;bheBcvh-oomuc:Ea’oyrdeapIaMoj arYT-,ealddmsRnmTymThAnkuaendethamhoaytenTdeeaanAnJ
TpteifmusDmnlndmecrmeaoakreaeIooue
Diversity: 0.5
e
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To FbareaAejdpCkm
;zEzhro—kYo—lvmYB;Jnswke!e!zqwkdt’dIRlRgmFhnblSRY.asuubkFakI-MDssd;DaihiTt ORmbOhes:ffFA  tORsCewm;Sc:Ah’AToOpOzoffskf’lmhsoaT:adyhvOligzA,!;s.pskBcpetC-pnpesmpcvFdAujlen;IsrjSuqnykhAlCmhiqb-.vy
DabuoY!qY tnsl
T’Hw,.vCqAmbtonwde yOBRotlteag-bOawHAJ:’dMTivD a,uz

In [ ]:
def generate_text(length,diversity):

  start_index = random.randint(0,len(poem)-max_length - 1)
  generated_text = ""
  sentence = poem[start_index: start_index+max_length]
  generated_text += sentence

  for i in range(length):
    x_pred = np.zeros((1,max_length,len(vocab)))

    for t,char in enumerate(sentence):
      x_pred[0,t,char_to_indices[char]] = 1

    preds = model.predict(x_pred,verbose=0)[0]
    next_index = sample_index(preds,diversity)

    next_char = indices_to_char[next_index]
    generated_text += next_char

    sentence = sentence[1:]+next_char

  return generated_text